# Hacker News Upvote Prediction: Exploratory Data Analysis (EDA)

Welcome to the EDA phase of the Hacker News Upvote Prediction project! This notebook shows how to:

1. Connect to the database and extract a 100k-row sample of posts
2. Inspect and visualize the data for outliers, distributions, and basic relationships
3. Prompt deeper investigation by posing guiding questions

We'll use **matplotlib** for plots, **pandas** for data manipulation, and **sqlalchemy** (or `psycopg2`) for database connections.

> **Tip**: If you have a local file or already-downloaded data, you can skip the database connection and simply load the data directly.

In [ ]:
# -- Install any missing libraries (run in separate terminal or comment out as needed)
# !pip install pandas sqlalchemy psycopg2 matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

# If you want inline plots in Jupyter, uncomment:
%matplotlib inline

print('Libraries imported successfully!')


## 1. Database Connection & Data Extraction
Here, we connect to the Postgres database and sample about 100,000 rows.

### Strategic Questions
- **How** did you decide on 100k? Could you choose more or fewer rows?
- **Do you need** to randomize the selection for representativeness?
- **Is the `title` column** ever missing (NULL)? Should you filter on that?


In [ ]:
# Example connection string. Replace with your actual credentials:
USERNAME = 'YOUR_USERNAME'
PASSWORD = 'YOUR_PASSWORD'
HOST     = 'YOUR_HOST'
PORT     = 'YOUR_PORT'  # e.g. 5432
DBNAME   = 'YOUR_DBNAME'

connection_string = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}"
engine = create_engine(connection_string)

# Adjust schema/table names to match your environment
QUERY = '''
SELECT *
FROM "hacker_news"."items"
WHERE title IS NOT NULL
ORDER BY random()
LIMIT 100000;
'''

df = pd.read_sql(QUERY, engine)
print(f"DataFrame shape: {df.shape}")
df.head()

## 2. Basic Data Inspection
Get a feel for what columns are available, data types, and missing values.

### Strategic Questions
- **Which columns** do you expect to be most predictive of upvotes?
- **Which columns** might need cleaning (e.g., domain, author)?
- **Are there any** columns that are entirely null or rarely populated?


In [ ]:
# Info and summary stats
df.info()
df.describe(include="all")

In [ ]:
# Check for nulls or missing data
missing_counts = df.isnull().sum()
print("Missing values in each column:\n", missing_counts)

# If you have time columns, convert them if needed:
# e.g., if 'time' is a UNIX timestamp, do:
# df['time'] = pd.to_datetime(df['time'], unit='s')

## 3. Univariate Analysis
Let's look at individual distributions for columns like **score** (upvotes), **title length**, etc.

### Strategic Questions
- **Is the score distribution** highly skewed? Any outliers?
- **Do you plan** a log transform for `score`?
- **What is** the typical length of a Hacker News title?


In [ ]:
# Score distribution
plt.hist(df['score'].dropna(), bins=50)
plt.title('Distribution of Hacker News Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Title length distribution
df['title_length'] = df['title'].apply(lambda x: len(str(x).split()))

plt.hist(df['title_length'], bins=50)
plt.title('Distribution of Title Length')
plt.xlabel('Number of Words in Title')
plt.ylabel('Frequency')
plt.show()


## 4. Author & Domain Exploration
We might suspect that certain authors consistently get higher upvotes or that certain domains do better.

### Strategic Questions
- **Which authors** appear most frequently?
- **Do they** also get higher scores on average?
- **Any domain** extremely popular or relevant?


In [ ]:
# Top 20 authors by frequency
top_authors = df['author'].value_counts().head(20)
print("Top 20 authors:\n", top_authors)

# Maybe examine average score for these authors
df_top_authors = df[df['author'].isin(top_authors.index)]
author_avg_scores = df_top_authors.groupby('author')['score'].mean().sort_values(ascending=False)
print("\nAverage scores for top authors:\n", author_avg_scores)

In [ ]:
# Domain frequency (if 'domain' column exists)
if 'domain' in df.columns:
    top_domains = df['domain'].value_counts().head(20)
    print("Top 20 domains:\n", top_domains)
else:
    print("No 'domain' column found.")

## 5. Bivariate Analysis
Let's look at relationships between pairs of variables.

### Strategic Questions
- **Does the length of the title** correlate with the score?
- **Is there** any time trend (older posts vs. newer posts)?
- **Do high scores** cluster around certain domains or authors?


In [ ]:
# Score vs. title length scatter
plt.scatter(df['title_length'], df['score'], alpha=0.3)
plt.title('Score vs. Title Length')
plt.xlabel('Title Length (words)')
plt.ylabel('Score')
plt.show()

In [ ]:
# If there's a time column, see how score changes over time
if 'time' in df.columns:
    # Convert time to datetime if needed
    # df['time'] = pd.to_datetime(df['time'], unit='s')  # uncomment if in UNIX format

    df['year_month'] = df['time'].dt.to_period('M')
    monthly_scores = df.groupby('year_month')['score'].mean().reset_index()

    # Convert year_month to string for plotting
    x_vals = monthly_scores['year_month'].astype(str)
    y_vals = monthly_scores['score']

    plt.plot(x_vals, y_vals)
    plt.title('Average Score by Year-Month')
    plt.xlabel('Date (Year-Month)')
    plt.ylabel('Average Score')
    plt.xticks(rotation=45)
    plt.show()
else:
    print("No 'time' column found for time-based analysis.")

## 6. Correlation & Outliers
For numeric columns, let's quickly generate a correlation matrix. We'll also look for outliers in `score`.

### Strategic Questions
- **Which features** have the highest correlation with `score`?
- **Do any** columns have surprisingly high or low correlation?
- **Should you** remove or transform outliers in `score`?


In [ ]:
# Create a subset of numeric columns for correlation
numeric_cols = ['score', 'title_length']
cols_in_df = [col for col in numeric_cols if col in df.columns]

if cols_in_df:
    corr_matrix = df[cols_in_df].corr()
    print("Correlation matrix:\n", corr_matrix)

    # Plot with matplotlib's imshow
    fig, ax = plt.subplots()
    cax = ax.imshow(corr_matrix)
    ax.set_xticks(range(len(cols_in_df)))
    ax.set_yticks(range(len(cols_in_df)))
    ax.set_xticklabels(cols_in_df)
    ax.set_yticklabels(cols_in_df)
    plt.colorbar(cax)
    plt.title('Correlation Heatmap')
    plt.show()
else:
    print("No numeric columns found for correlation analysis.")

In [ ]:
# Quick look at potential outliers in 'score'
score_sorted = df['score'].sort_values(ascending=False)
print("Top 10 highest scores:")
print(score_sorted.head(10))

## 7. Insights, Questions, and Next Steps
### Observations
- Are there extremely **high scores** that might skew your model?
- Are some authors or domains **dominating** the dataset?
- Does the data **span many years** and require a time-based train/test split?

### Potential Next Steps
1. **Data Cleaning**: Fix missing or null values, unify domain formats (e.g., remove `"www."`).
2. **Feature Engineering**: 
   - Possibly create a `log_score` to reduce the effect of large outliers.
   - Encode author/domain for the model (one-hot or embedded).
3. **Time-based Split** (if relevant): Train on older data, test on newer.
4. **Word2Vec**: Start building a pipeline to embed titles.

**Feel free** to expand this notebook with deeper analysis, e.g., comparing top 10 authors over time, or investigating comment counts if available.

> **Reminder**: The main goal is to understand the data before jumping into modeling so that you can make informed decisions on feature selection and model architecture.